# LAI trend over China

## Import libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import regionmask
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download

plt.style.use("seaborn-v0_8-notebook")

## Set parameters

In [ ]:
year_start = 2001
year_stop = 2019

## Define requests

In [ ]:
collection_id = "satellite-lai-fapar"
base_request = {
    "variable": "lai",
    "sensor": "vgt",
    "horizontal_resolution": "1km",
    "product_version": "v3",
    "area": [53, 73, 18, 135],
}

requests = []
for time in pd.date_range(str(year_start), str(year_stop + 1), freq="ME"):
    requests.append(
        base_request
        | {
            "satellite": "spot" if time.year <= 2013 else "proba",
            "year": str(time.year),
            "month": f"{time.month:02d}",
            "nominal_day": ["10", "20", str(time.day)],
        }
    )

## Define functions to cache

In [ ]:
def spatial_weighted_mean_over_china(ds):
    regions = regionmask.defined_regions.natural_earth_v5_0_0.countries_110

    spatial_mask = regions[regions.map_keys(["China"])].mask(ds).notnull()
    flag_mask = (ds["retrieval_flag"].astype("int") & 0xFC1) == 0
    lai_mask = ds["LAI"] < 65_535

    ds = ds[["LAI"]].where(spatial_mask & flag_mask & lai_mask)
    return diagnostics.spatial_weighted_mean(ds)

## Download and transform

In [ ]:
ds = download.download_and_transform(
    collection_id,
    requests,
    transform_func=spatial_weighted_mean_over_china,
)
da = ds["LAI"].groupby("time.year").mean()

## Plot timeseries

In [ ]:
coeffs = da.polyfit(dim="year", deg=1)
da_fit = xr.polyval(da["year"], coeffs)["polyfit_coefficients"]
da_fit.attrs = da.attrs
equation = (
    "y ="
    f" {coeffs.polyfit_coefficients.sel(degree=1).values:.2f}x"
    f" {coeffs.polyfit_coefficients.sel(degree=0).values:+.2f}"
)
da.plot(ls="--", marker="o", label="LAI")
da_fit.plot(label=equation)
plt.grid()
plt.legend()
_ = plt.title("LAI Trend Over Selected Years")